In [20]:
# PyTorch 라이브러리 import  *** 해당 cell을 수정하지 말 것 ***
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [26]:
# TODO: CIFAR-100 training set 불러오기
import random

random.seed(2024)
torch.manual_seed(2024)
torch.cuda.manual_seed_all(2024)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor()
])

batch_size = 128
train_dataset = datasets.CIFAR100(root='.', train=True, download=True,
                                  transform=transform_train)
# train_size = int(0.8 * len(train_dataset))
# validation_size = len(train_dataset) - train_size
# train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, validation_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           shuffle=True, num_workers=2)
# validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size,
#                                                 shuffle=False, num_workers=2)

Files already downloaded and verified


In [30]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        torch.manual_seed(2024)  # 결과 재현을 위한 seed number 고정 *** 해당 line을 수정하지 말 것 ***
        
        # TODO: CNN 구성 layer들 선언
        
        # 32x32x3 -> 16x16x64
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.bn1_1 = nn.BatchNorm2d(64)
        
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn1_2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 16x16x64 -> 8x8x128
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2_1 = nn.BatchNorm2d(128)
        
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn2_2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 8x8x128 -> 4x4x256
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3_1 = nn.BatchNorm2d(256)
        
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn3_2 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # flatten
        self.flatten = nn.Flatten()
        
        # fcl
        self.fc4 = nn.Linear(4 * 4 * 256, 4096)
        self.bn4 = nn.BatchNorm1d(4096)
        self.dropout4 = nn.Dropout(0.6)
        self.fc5 = nn.Linear(4096, 4096)
        self.bn5 = nn.BatchNorm1d(4096)
        self.dropout5 = nn.Dropout(0.6)
        self.fc6 = nn.Linear(4096, 100)
        
        # relu
        self.relu = nn.ReLU(inplace=True)

        # xavier initialization
        nn.init.xavier_uniform_(self.conv1_1.weight)
        nn.init.xavier_uniform_(self.conv1_2.weight)
        nn.init.xavier_uniform_(self.conv2_1.weight)
        nn.init.xavier_uniform_(self.conv2_2.weight)
        nn.init.xavier_uniform_(self.conv3_1.weight)
        nn.init.xavier_uniform_(self.conv3_2.weight)
        nn.init.xavier_uniform_(self.fc4.weight)
        nn.init.xavier_uniform_(self.fc5.weight)
        
    def forward(self, x):
        # TODO: forward pass 정의
        x = self.relu(self.bn1_1(self.conv1_1(x)))
        x = self.relu(self.bn1_2(self.conv1_2(x)))
        x = self.pool1(x)

        x = self.relu(self.bn2_1(self.conv2_1(x)))
        x = self.relu(self.bn2_2(self.conv2_2(x)))
        x = self.pool2(x)

        x = self.relu(self.bn3_1(self.conv3_1(x)))
        x = self.relu(self.bn3_2(self.conv3_2(x)))
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)
        x = self.relu(self.bn5(self.fc5(x)))
        x = self.dropout5(x)
        x = self.fc6(x)

        return x


model = CNN().cuda()

num_epochs = 10
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

In [31]:
# TODO: model 명의로 생성된 CNN 모델에 대해 학습 수행하기
for epoch in range(num_epochs):
    total = 0
    correct = 0
    train_loss = 0.0

    # training
    model.train()
    print(f'        Epoch [{epoch+1}/{num_epochs}]')
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100. * correct / total

#     # validation
#     model.eval()
#     val_loss = 0.0
#     val_total = 0
#     val_correct = 0
#     with torch.no_grad():
#         for images, labels in validation_loader:
#             images, labels = images.to('cuda'), labels.to('cuda')
#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             val_loss += loss.item()
#             _, predicted = outputs.max(1)
#             val_total += labels.size(0)
#             val_correct += predicted.eq(labels).sum().item()

#     val_loss /= len(validation_loader)
#     val_accuracy = 100. * val_correct / val_total

    print(f'     Train Loss: {train_loss:.4f}')
    #print(f'Validation Loss: {val_loss:.4f}')
    print(f'      train_acc: {train_accuracy:.2f}%\n')
    #print(f'        val_acc: {val_accuracy:.2f}%\n')

    scheduler.step()

        Epoch [1/10]
     Train Loss: 4.5404
      train_acc: 5.85%

        Epoch [2/10]
     Train Loss: 3.8492
      train_acc: 14.37%

        Epoch [3/10]
     Train Loss: 3.3049
      train_acc: 21.17%

        Epoch [4/10]
     Train Loss: 2.9016
      train_acc: 27.22%

        Epoch [5/10]
     Train Loss: 2.6520
      train_acc: 32.78%

        Epoch [6/10]
     Train Loss: 2.4412
      train_acc: 36.88%

        Epoch [7/10]
     Train Loss: 2.2555
      train_acc: 40.81%

        Epoch [8/10]
     Train Loss: 2.0589
      train_acc: 45.33%

        Epoch [9/10]
     Train Loss: 1.8948
      train_acc: 49.22%

        Epoch [10/10]
     Train Loss: 1.7726
      train_acc: 52.14%



In [32]:
# 학습된 모델 평가  *** 해당 cell을 수정하지 말 것 ***


test_dataset = datasets.CIFAR100(root='.', train=False, download=True,
                                 transform=transforms.ToTensor())
batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image = image.cuda()
    label = label.cuda()
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')

Files already downloaded and verified
Accuracy : 55.82 %
